In [1]:
from quantrocket import get_prices
import pandas as pd
import numpy as np

In [2]:
# Step 1: Fetch daily close prices of Apple stock for the year 2023
start_date = '2023-01-01'
end_date = '2023-12-31'
prices = get_prices("usstock-free-1d", universes="usstock-free", start_date=start_date, end_date=end_date, fields=["Close"])
close_prices = prices['FIBBG000B9XRY4']

print('\n',close_prices)


 Field  Date      
Close  2023-01-03    124.2163
       2023-01-04    125.4975
       2023-01-05    124.1666
       2023-01-06    128.7352
       2023-01-09    129.2616
                       ...   
       2023-12-22    193.3533
       2023-12-26    192.8040
       2023-12-27    192.9038
       2023-12-28    193.3333
       2023-12-29    192.2846
Name: FIBBG000B9XRY4, Length: 250, dtype: float64


In [3]:
# Step 2: Calculate daily returns and classify states
returns = close_prices.pct_change()
print(returns)
states = np.where(returns >= 0.01, 1, np.where(returns > -0.01, 0, -1))
print(states)

Field  Date      
Close  2023-01-03         NaN
       2023-01-04    0.010314
       2023-01-05   -0.010605
       2023-01-06    0.036794
       2023-01-09    0.004089
                       ...   
       2023-12-22   -0.005547
       2023-12-26   -0.002841
       2023-12-27    0.000518
       2023-12-28    0.002226
       2023-12-29   -0.005424
Name: FIBBG000B9XRY4, Length: 250, dtype: float64
[-1  1 -1  1  0  0  1  0  1  0  0  0  1  1  1  0  1  1 -1  0  0  1  1 -1
  1 -1  0  0  1  0  1 -1  0 -1  0  0 -1  0  0 -1  0  1  1 -1  0 -1 -1  1
  1  0  1  0  1  1  0  0  0 -1  0  1  0  1  0  0 -1  0 -1  0  0  1  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  1  0  0  0  0  0  1  0
  0 -1  0  0  1  1  0  1  0  0  0  0  1  0  1  0  0  1  0  0  0  1  0  0
  1  0  0  1  0  0  0  0 -1  0  0  0  0  1  0  0 -1  0  0  0  0  0  1  0
  0 -1  0 -1 -1  0  0  0  0  0 -1  0 -1  0  0  0  1 -1  1  0  1  1  0  0
  0 -1 -1  0  0 -1 -1  0  0  1  0 -1  0  0  0 -1  0  0  0  1  0  0  0  1
  0  0  0  0 -1  0

In [4]:
# Step 3: Implement the value function
portfolio_value = 0
state_transitions = np.zeros((3, 3))
optimal_buy_indices = []

for i in range(1, len(states)):
    if states[i] == 1 and states[i-1] == 0:
        portfolio_value += 1
        optimal_buy_indices.append(i-1)
    elif states[i] == -1 and states[i-1] == 0:
        portfolio_value -= 1

    state_transitions[states[i-1] + 1, states[i] + 1] += 1

In [5]:
# Step 4: Calculate transition distribution
transition_distribution = np.zeros_like(state_transitions)
# Calculate transition distribution, handling division by zero
for i in range(len(state_transitions)):
    row_sum = np.sum(state_transitions[i])
    if row_sum != 0:
        transition_distribution[i] = state_transitions[i] / row_sum
print(transition_distribution)

[[0.13888889 0.72222222 0.13888889]
 [0.14649682 0.59872611 0.25477707]
 [0.125      0.67857143 0.19642857]]


In [7]:
print("Portfolio Value V(d):", portfolio_value)
print("Optimal Index:",optimal_buy_indices)
print("Transition Distribution:")
print(pd.DataFrame(transition_distribution, index=['Bear', 'Flat', 'Bull'], columns=['Bear', 'Flat', 'Bull']))

Portfolio Value V(d): 17
Optimal Index: [5, 7, 11, 15, 20, 27, 29, 40, 49, 51, 58, 60, 68, 78, 84, 87, 93, 99, 102, 107, 109, 112, 116, 119, 122, 132, 141, 159, 163, 176, 186, 190, 206, 208, 211, 215, 217, 231, 233, 237]
Transition Distribution:
          Bear      Flat      Bull
Bear  0.138889  0.722222  0.138889
Flat  0.146497  0.598726  0.254777
Bull  0.125000  0.678571  0.196429
